In [33]:
''' LIBRARIES '''
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import time
from IPython.display import clear_output

from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve
import pandas as pd
import numpy as np

# ----------------------------------------------

# Merge Starbet / Admiralbet

In [34]:
df_star, df_adm = pd.read_csv('CSV\df_star.csv') , pd.read_csv('CSV\df_adm.csv')

teams_adm  = df_adm.teams.tolist()
teams_star = df_star.teams.tolist()

df_adm[['teams_matched', 'score']] = df_adm.teams.apply(lambda x:process.extractOne(x, teams_star, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
df_adm = df_adm[df_adm.score>60]

df_star_adm = pd.merge(df_adm, df_star, left_on='teams_matched', right_on='teams', how='left')
df_star_adm.to_csv('CSV\TwoWay\PRE_star_adm.csv')
df_star_adm[['teams_x','teams_matched', 'score']]

,teams_x,teams_matched,score
0,Cobreloa Deportes Magallanes,Cobreloa Magallanes,81
1,FBC Melgar Binacional,FBC Melgar Deportivo Binacional,81
2,3 de Febrero Sportivo Iteno,3 de Febrero Sportivo Iteno,100
3,Atletico Morelia Tepatitlan de Morelos,Atl. Morelia Tepatitlan de Morelos,93
4,Tombense Novorizontino,Tombense Gremio Novorizontino,86
...,...,...,...
60,ASV Drassburg Kremser,Drassburg Kremser SC,83
61,Aalborg Odense,Aalborg Odense,100
62,Aue Meppen,Aue Meppen,100
63,Deutschlandsberger Bad Gleichenberg,Deutschlandsberger Bad Gleichenberg,100


# -----------------------------------
# Merge Admiral / Soccerbet

In [35]:
df_adm, df_soc = pd.read_csv('CSV\df_adm.csv'), pd.read_csv('CSV\df_soc.csv')

teams_adm = df_adm.teams.tolist()
teams_soc = df_soc.teams.tolist()

df_soc[['teams_matched', 'score']] = df_soc.teams.apply(lambda x: process.extractOne(x, teams_adm, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
df_soc = df_soc[df_soc.score>50]

df_soc_adm = pd.merge(df_soc, df_adm, left_on='teams_matched', right_on='teams', how='left')
df_soc_adm.to_csv('CSV\TwoWay\PRE_soc_adm.csv')
df_soc_adm

,Unnamed: 0_x,teams_x,1_soc,X_soc,2_soc,under_soc,goals_soc,over_soc,teams_matched,score,Unnamed: 0_y,teams_y,1_adm,X_adm,2_adm,under_adm,goals_adm,over_adm
0,0,Cobreloa Magallanes,2.00,3.15,4.05,1.50,2.5,2.45,Cobreloa Deportes Magallanes,81,0,Cobreloa Deportes Magallanes,1.95,3.00,4.30,1.45,2.5,2.55
1,1,FBC Melgar Dep.Binacional,1.30,4.75,9.50,1.92,2.5,1.78,FBC Melgar Binacional,91,1,FBC Melgar Binacional,1.30,5.10,9.50,2.02,2.5,1.73
2,2,3 de Febrero Sportivo Iteno,1.47,3.95,5.70,1.95,2.5,1.75,3 de Febrero Sportivo Iteno,100,5,3 de Febrero Sportivo Iteno,1.49,4.10,5.70,2.02,2.5,1.70
3,3,Tombense Novorizontino,2.85,2.80,2.60,1.40,2.5,2.70,Tombense Novorizontino,100,8,Tombense Novorizontino,2.95,2.80,2.65,1.38,2.5,2.85
4,4,Atletico Morelia Tepatitlan,1.65,3.70,4.90,1.82,2.5,1.88,Atletico Morelia Tepatitlan de Morelos,83,7,Atletico Morelia Tepatitlan de Morelos,1.66,3.70,5.00,1.88,2.5,1.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,66,Z Lippstadt Ahlen,2.30,3.30,2.65,2.10,2.5,1.65,Lippstadt Ahlen,94,140,Lippstadt Ahlen,2.40,3.45,2.60,2.05,2.5,1.68
76,67,Al Quds Shabab Al Khaleel,3.40,3.25,1.95,1.90,2.5,1.80,Hilal Al Quds Shabab Al Khaleel,89,115,Hilal Al Quds Shabab Al Khaleel,4.30,3.65,1.71,1.95,2.5,1.76
77,67,Al Quds Shabab Al Khaleel,3.40,3.25,1.95,1.90,2.5,1.80,Hilal Al Quds Shabab Al Khaleel,89,138,Hilal Al Quds Shabab Al Khaleel,4.30,3.65,1.71,1.95,2.5,1.76
78,68,Z Lippstadt Ahlen,2.30,3.30,2.65,2.10,2.5,1.65,Lippstadt Ahlen,94,117,Lippstadt Ahlen,2.40,3.45,2.60,2.05,2.5,1.68


# ----------------------------------------------

# Merge Starbet / Soccerbet

In [36]:
df_star, df_soc = pd.read_csv('CSV\df_star.csv'), pd.read_csv('CSV\df_soc.csv')

teams_star = df_star.teams.tolist()
teams_soc = df_soc.teams.tolist()

df_soc[['teams_matched', 'score']] = df_soc.teams.apply(lambda x: process.extractOne(x, teams_star, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
df_soc = df_soc[df_soc.score>50]

df_soc_star = pd.merge(df_soc, df_star, left_on='teams_matched', right_on='teams', how='left')
df_soc_star.to_csv('CSV\TwoWay\PRE_soc_star.csv')
df_soc_star

,Unnamed: 0_x,teams_x,1_soc,X_soc,2_soc,under_soc,goals_soc,over_soc,teams_matched,score,Unnamed: 0_y,teams_y,1_star,X_star,2_star,under_star,goals_star,over_star
0,0,Cobreloa Magallanes,2.00,3.15,4.05,1.50,2.5,2.45,Cobreloa Magallanes,100,397,Cobreloa Magallanes,2.07,3.10,3.55,1.53,2.5,2.35
1,1,FBC Melgar Dep.Binacional,1.30,4.75,9.50,1.92,2.5,1.78,FBC Melgar Deportivo Binacional,89,515,FBC Melgar Deportivo Binacional,1.33,4.50,9.50,5.00,2.5,1.76
2,2,3 de Febrero Sportivo Iteno,1.47,3.95,5.70,1.95,2.5,1.75,3 de Febrero Sportivo Iteno,100,513,3 de Febrero Sportivo Iteno,1.47,3.95,5.70,1.95,2.5,1.70
3,3,Tombense Novorizontino,2.85,2.80,2.60,1.40,2.5,2.70,Tombense Gremio Novorizontino,86,386,Tombense Gremio Novorizontino,2.75,2.80,2.70,1.39,2.5,2.80
4,4,Atletico Morelia Tepatitlan,1.65,3.70,4.90,1.82,2.5,1.88,Atl. Morelia Tepatitlan de Morelos,77,491,Atl. Morelia Tepatitlan de Morelos,1.62,3.65,5.05,1.80,2.5,1.90
5,5,Santos DG Perez Zeledon,2.30,3.55,2.70,2.12,2.5,1.65,Santos Guápiles Perez Zeledon,86,401,Santos Guápiles Perez Zeledon,2.18,3.35,2.85,2.07,2.5,1.62
6,6,America Quito Imbabura,2.40,2.80,2.95,1.45,2.5,2.55,America de Quito Imbabura,94,441,America de Quito Imbabura,2.40,2.90,2.95,1.46,2.5,2.45
7,7,Tapatio Venados,2.30,3.10,3.05,1.62,2.5,2.15,CD Tapatio Venados Merida,75,492,CD Tapatio Venados Merida,2.25,3.10,3.10,1.60,2.5,2.20
8,8,Vasco da Gama Londrina,1.67,3.10,6.15,1.45,2.5,2.55,Vasco da Gama Londrina,100,387,Vasco da Gama Londrina,1.70,3.25,5.75,1.45,2.5,2.60
9,9,Ind. Santa Fe Atl. Nacional,2.65,3.20,2.65,1.70,2.5,2.03,Santa Fe Atl. Nacional,91,399,Santa Fe Atl. Nacional,2.60,3.15,2.60,1.69,2.5,2.05


# ----------------------------------------------

# Merge Maxbet / Starbet

In [37]:
df_star, df_max = pd.read_csv('CSV\df_star.csv'), pd.read_csv('CSV\df_max.csv')

teams_star = df_star.teams.tolist()
teams_max = df_max.teams.tolist()

df_star[['teams_matched', 'score']] = df_star.teams.apply(lambda x:process.extractOne(x, teams_max, scorer= fuzz.token_sort_ratio)).apply(pd.Series)
df_star = df_star[df_star.score>59]

df_star_max = pd.merge(df_star, df_max, left_on='teams_matched', right_on='teams', how='left')
df_star_max.to_csv('CSV\TwoWay\PRE_star_max.csv')



# ----------------------------------------------

# Merge Maxbet / Admiral

In [38]:
df_adm, df_max = pd.read_csv('CSV\df_adm.csv'), pd.read_csv('CSV\df_max.csv')

teams_adm = df_adm.teams.tolist()
teams_max = df_max.teams.tolist()

df_adm[['teams_matched', 'score']] = df_adm.teams.apply(lambda x:process.extractOne(x, teams_max, scorer= fuzz.token_sort_ratio)).apply(pd.Series)
df_adm = df_adm[df_adm.score>59]

df_adm_max = pd.merge(df_adm, df_max, left_on='teams_matched', right_on='teams', how='left')
df_adm_max.to_csv('CSV\TwoWay\PRE_adm_max.csv')
df_adm_max

,Unnamed: 0_x,teams_x,1_adm,X_adm,2_adm,under_adm,goals_adm,over_adm,teams_matched,score,Unnamed: 0_y,teams_y,1_max,X_max,2_max,under_max,goals_max,over_max
0,0,Cobreloa Deportes Magallanes,1.95,3.00,4.30,1.45,2.5,2.55,Cobreloa Magallanes,81,48,Cobreloa Magallanes,2.20,3.00,3.30,1.50,2.5,2.45
1,1,FBC Melgar Binacional,1.30,5.10,9.50,2.02,2.5,1.73,Melgar Binacional,89,157,Melgar Binacional,1.32,4.55,9.00,1.90,2.5,1.80
2,2,Lobos ULMX Gavilanes de Matamoros,2.55,3.40,2.50,1.91,2.5,1.79,Lobos ULMX Gavilanes de M.,86,139,Lobos ULMX Gavilanes de M.,2.45,3.25,2.55,1.95,2.5,1.70
3,3,Montaneses FC CF Pachuca II,3.80,3.90,1.75,2.65,2.5,1.42,Montaneses Pachuca 2,81,140,Montaneses Pachuca 2,3.80,3.80,1.70,2.70,2.5,1.38
4,4,Yalmakan Cafetaleros de Chiapas,3.15,3.50,2.05,2.07,2.5,1.67,Yalmakan Cafetaleros,78,141,Yalmakan Cafetaleros,3.05,3.50,2.00,2.05,2.5,1.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,163,Karlskrona Asarums,1.67,3.95,4.20,2.55,2.5,1.45,Karlskrona Asarum,97,173,Karlskrona Asarum,1.65,3.80,4.15,2.50,2.5,1.42
108,164,Kristianstads Hassleholms IF,2.85,3.45,2.22,2.22,2.5,1.58,Kristianstad Hassleholms IF,98,174,Kristianstad Hassleholms IF,2.80,3.35,2.20,2.20,2.5,1.55
109,165,Landvetter IS Frolunda,2.65,3.55,2.30,2.55,2.5,1.45,Landvetter Frolunda,93,169,Landvetter Frolunda,2.60,3.60,2.20,2.50,2.5,1.42
110,166,Motala Sylvia,2.20,3.50,2.85,2.35,2.5,1.52,Motala Sylvia,100,167,Motala Sylvia,2.25,3.50,2.75,2.30,2.5,1.50


# ----------------------------------------------

# Merge Maxbet / Soccerbet

In [39]:
df_soc, df_max = pd.read_csv('CSV\df_soc.csv'), pd.read_csv('CSV\df_max.csv')

teams_soc = df_soc.teams.tolist()
teams_max = df_max.teams.tolist()

df_soc[['teams_matched', 'score']] = df_soc.teams.apply(lambda x:process.extractOne(x, teams_max, scorer= fuzz.token_sort_ratio)).apply(pd.Series)
df_soc = df_soc[df_soc.score>59]

df_soc_max = pd.merge(df_soc, df_max, left_on='teams_matched', right_on='teams', how='left')
df_soc_max.to_csv('CSV\TwoWay\PRE_soc_max.csv')
df_soc_max

,Unnamed: 0_x,teams_x,1_soc,X_soc,2_soc,under_soc,goals_soc,over_soc,teams_matched,score,Unnamed: 0_y,teams_y,1_max,X_max,2_max,under_max,goals_max,over_max
0,0,Cobreloa Magallanes,2.00,3.15,4.05,1.50,2.5,2.45,Cobreloa Magallanes,100,48,Cobreloa Magallanes,2.20,3.00,3.30,1.50,2.5,2.45
1,1,FBC Melgar Dep.Binacional,1.30,4.75,9.50,1.92,2.5,1.78,Melgar Binacional,81,157,Melgar Binacional,1.32,4.55,9.00,1.90,2.5,1.80
2,2,3 de Febrero Sportivo Iteno,1.47,3.95,5.70,1.95,2.5,1.75,3 De Febrero Sp.Iteno,88,149,3 De Febrero Sp.Iteno,1.48,3.95,5.70,1.98,2.5,1.68
3,3,Tombense Novorizontino,2.85,2.80,2.60,1.40,2.5,2.70,Tombense Novorizontino,100,2,Tombense Novorizontino,2.00,3.00,3.00,2.90,2.5,1.48
4,3,Tombense Novorizontino,2.85,2.80,2.60,1.40,2.5,2.70,Tombense Novorizontino,100,41,Tombense Novorizontino,2.80,2.80,2.70,1.38,2.5,2.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,63,Dnepr Mogilev Slutsk,3.60,3.40,1.95,1.80,2.5,1.90,D.Mogilev Slutsk,89,35,D.Mogilev Slutsk,3.45,3.30,2.00,1.80,2.5,1.90
58,65,Sabail Qabala,3.70,3.20,2.00,1.65,2.5,2.10,Sabail Gabala,92,33,Sabail Gabala,3.25,2.90,2.15,1.58,2.5,2.12
59,66,Z Lippstadt Ahlen,2.30,3.30,2.65,2.10,2.5,1.65,Lippstadt Ahlen,94,112,Lippstadt Ahlen,2.45,3.45,2.55,2.05,2.5,1.68
60,67,Al Quds Shabab Al Khaleel,3.40,3.25,1.95,1.90,2.5,1.80,Al Quds Al Khaleel,84,153,Al Quds Al Khaleel,3.45,3.20,1.98,1.85,2.5,1.78


#       3 Way Merges

In [43]:
''' AD / ST / SC '''
# loading data
df_soc_adm = pd.read_csv('CSV\TwoWay\PRE_soc_adm.csv')
df_soc_adm.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Unnamed: 0_y','teams_x', 'teams_y', 'score' ], inplace=True)
df_star = pd.read_csv('CSV\df_star.csv')
#Start Merging process
teams_star = df_star.teams.tolist()
df_soc_adm[['teams_matched2', 'score']] = df_soc_adm.teams_matched.apply(lambda x:process.extractOne(x, teams_star, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
df_soc_adm = df_soc_adm[df_soc_adm.score>59] #This drops all the teams names that don't match among sites
#The last merging process
df_soc_adm_star = pd.merge(df_soc_adm, df_star, left_on='teams_matched2', right_on='teams', how='left')
df_soc_adm_star.to_csv('CSV\ThreeWay\Pre_soc_adm_star.csv')
print(df_soc_adm_star.shape)

(48, 23)


In [41]:
''' AD / SC / MAX '''
# loading data
df_soc_adm = pd.read_csv('CSV\TwoWay\PRE_soc_adm.csv')
df_soc_adm.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Unnamed: 0_y','teams_x', 'teams_y', 'score' ], inplace=True)
df_max = pd.read_csv('CSV\df_max.csv')
#Start Merging process
teams_max = df_max.teams.tolist()
df_soc_adm[['teams_matched2', 'score']] = df_soc_adm.teams_matched.apply(lambda x:process.extractOne(x, teams_max, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
df_soc_adm = df_soc_adm[df_soc_adm.score>59] #This drops all the teams names that don't match among sites
#The last merging process
df_soc_adm_max = pd.merge(df_soc_adm, df_max, left_on='teams_matched2', right_on='teams', how='left')
df_soc_adm_max.to_csv('CSV\ThreeWay\Pre_soc_adm_max.csv')
df_soc_adm_max.shape

(75, 23)

In [42]:
''' SC / ST / MAX '''
df_soc_star = pd.read_csv('CSV\TwoWay\PRE_soc_star.csv')
df_soc_star.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Unnamed: 0_y','teams_x', 'teams_y', 'score' ], inplace=True)

teams_max = df_max.teams.tolist()
df_soc_star[['teams_matched2', 'score']] = df_soc_star.teams_matched.apply(lambda x:process.extractOne(x, teams_max, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
df_soc_star = df_soc_star[df_soc_star.score>59] #This drops all the teams names that don't match among sites
#The last merging process
df_soc_star_max = pd.merge(df_soc_star, df_max, left_on='teams_matched2', right_on='teams', how='left')
df_soc_star_max.to_csv('CSV\ThreeWay\Pre_soc_star_max.csv')
df_soc_star_max.shape

(50, 23)